# Cash-Karp Runge-Kutta coupled ODE solver

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Define the coupled derivatives we need to integrate

In [ ]:
def dydx(x,y):
    #original diffeq is d^2y/dx^2 = -y
    
    #write as two coupled ODEs
        #dy/dx = z
        #dz/dx = -y
        
    #set y = y[0]
    #set z = y[1]
    y_derivs = np.zeros(2)
    y_derivs[0] = y[1] #dy/dx = z 
    y_derivs[1] = -1 * y[0] # dz/dx = -y
    
    return y_derivs

Build the 5th order RK Method

In [ ]:
def rk5_mv_core(dydx,xi,yi,nv,h):
    
    #declare k arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #set specific coefficients such that our error will be embedded in our formula and not require
    #extraneous calculations
    
    a_2 = 1/5.
    a_3 = 3/10.
    a_4 = 3/5.
    a_5 = 1.
    a_6 = 7/8.
    
    b_21 = 1/5.
    
    b_31 = 3/40.
    b_32 = 9/40.
    
    b_41= 3/10.
    b_42 = -1 * 9/10.
    b_43 = 6/5.
    
    b_51 = -1 * 11/54.
    b_52 = 5/2.
    b_53 = -70/27.
    b_54 = 35/27.
    
    b_61 = 1631/55296.
    b_62 = 175/512.
    b_63 = 575/13824.
    b_64 = 44275/110592.
    b_65 = 253/5096.
    
    c_1 = 37/378.
    c_2 = 0
    c_3 = 250/621.
    c_4 = 125/594.
    c_5 = 0
    c_6 = 512/1771.
    
    c_10 = 2825/27648.
    c_20 = 0
    c_30 = 18575/48384.
    c_40 = 13525/55296.
    c_50 = 277/14336.
    c_60 = 1/4.
    
    #declare a temporary y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_derivs = dydx(xi + a_2*h , yi[:] + b_21*k1[:])
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_derivs = dydx(xi + a_3*h, yi[:] + b_32*k2[:] + b_31*k1[:])
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_derivs = dydx(xi + a_4*h, yi[:] + b_43*k3[:] + b_42*k2[:] + b_41*k1[:])
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_derivs = dydx(xi + a_5*h, yi[:] + b_54*k4[:] + b_53*k3[:] + b_52*k2[:] + b_51*k1[:])
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    y_derivs = dydx(xi + a_6*h, yi[:] + b_65*k5[:] + b_64*k4[:] + b_63*k3[:] + b_62*k2[:] + b_61*k1[:]) 
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    
    #5th order solution
    yipo = yi + c_1*k1 + c_2*k2 + c_3*k3 + c_4*k4 + c_5*k5 + c_6*k6
    
    #embedded 4th order solution
    yipo0 = yi + c_10*k1 + c_20*k2 + c_30*k3 + c_40*k4 + c_50*k5 + c_60*k6
    
    return yipo, yipo0

Build driver for RK5 with adaptive step size

In [ ]:
def rk5_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #max iteration variable
    imax= 1000
    
    #declare iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv, 2*tol)
    
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        y_2 = rk5_mv_core(dydx, x_i, y_i, nv, h_step)[0]
        y_20 = rk5_mv_core(dydx, x_i, y_i, nv, h_step)[1]
        
        #compute an error
        
        #with these coefficients, our error estimate is 5th order soln - 4th order soln
        Delta = y_2 - y_20
        
        #if error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check the iteration
        if(i>=imax):
            print("Too many iterations in rk5_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        #iterate
        i+=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

Build wrapper function for RK5

In [ ]:
def rk5_mv(dydx,a,b,y_a,tol):
    
    #define our starting step
    xi = a
    yi = y_a.copy()

    #an initial (very small) step
    h = 1.0e-4 * (b-a)
    
    imax = 1000
    
    i = 0
    
    #set the number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv), y_a)
    
    #print (x)
    #print (y)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag == 1):

        #calculate y_i+1
        yi_new, h_new, h_step = rk5_mv_ad(dydx, xi, yi, nv, h, tol)
        
        #update the step
        h = h_new
        
        #stop if overshot boundary
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk5_mv_ad(dydx, xi, yi, nv, h, tol)

            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        if(xi==b):
            flag = 0
    
    return x,y

# Perform the Integration

NOTE: Here, I receive a numerical runtime warning that I could not identify. However, this does not stop the notebook from runnning.

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2
tol = 1.0e-6

x,y = rk5_mv(dydx, a, b, y_0, tol)
print(x)

# Plot the numerical result and the analytical solution

In [ ]:
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx, np.sin(xx), linewidth=2, label = "sin(x)", color = "orange")
plt.plot(xx, np.cos(xx), linewidth=2, label = "cos(x)",color = "green")

plt.plot(x,y[:,0], label = 'y(x)', color = "blue")
plt.plot(x,y[:,1], label = 'dy/dx', color = "red")



plt.xlabel("x")
plt.ylabel("y, dy/dx")
plt.legend(frameon=False)

# Plot the error

In [ ]:
sin = np.sin(x)
cos = np.cos(x)

y_err = (y[:, 0]-sin)
dydx_err = (y[:,1]-cos)

plt.plot(x, y_err, label = "y(x) Error")
plt.plot(x, dydx_err, label = "dy/dx Error")
plt.legend(frameon=False)

# Plot the absolute error

In [ ]:
sin = np.sin(x)
cos = np.cos(x)

y_err = (np.fabs(y[:, 0]-sin))
dydx_err = (np.fabs(y[:,1]-cos))

plt.plot(x, y_err, label = "y(x) Error")
plt.plot(x, dydx_err, label = "dy/dx Error")
plt.legend(frameon=False)